In [1]:
# IMPORT ALL THE NECESSARY LIBRARIES
#
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import r2_score
import sys
import os
#
# IDENTIFY THE DEFAULT PATH FOR ALL OUTPUT FILES (change the variable for path_1 to whatever your filepath is):
#
path_1 = Path(r'C:\Users\zavarin1\Documents\6. DRESDEN SMART KD CALCS\jaea_online_extract')
#path_1=Path(r'C:\Users\rk260\OneDrive\Desktop\japan')
#
# Pull in datasets, tab delimited
#
df = pd.read_csv(path_1.joinpath("SDBDataDownload.txt"),sep = '\t',low_memory=False)
#print(df.head(2))

In [2]:
# Pull in dataset2, tab delimited
#
df1 = pd.read_csv(path_1.joinpath("SDBDataDownload-1.txt"),sep = '\t',low_memory=False)
#print(df1.head(2))

In [3]:
# Pull in dataset3, tab delimited
#
df2 = pd.read_csv(path_1.joinpath("SDBDataDownload-2.txt"),sep = '\t',low_memory=False)
#print(df2.head(2))

In [4]:
# Pull in dataset4, tab delimited
#
df3 = pd.read_csv(path_1.joinpath("SDBDataDownload-3.txt"),sep = '\t',low_memory=False)
#print(df3.head(2))

In [5]:
frames = [df, df1, df2, df3]
#df4 = pd.concat(frames)
#print(df4.head(2))



In [6]:
#Select subset of solid phase groups for further analysis and export csv file
df5 = pd.concat(frames)
acceptable=['Other minerals', 'Clay minerals', 'Bentonite (smectite)']
indexNames = df5[ ~df5['Solid Phase Group'].isin(acceptable) ].index
df5.drop(indexNames, inplace=True)
#df5.to_csv('df5.csv')

In [7]:
#Export csv file of all atm/redox conditions in the jaea subset file
atm_conditions = list(set(df5["atm/redox condition"]))
df10 = pd.DataFrame(atm_conditions)
df10.to_csv(path_1.joinpath('atm_conditions.csv'), index = False)

In [8]:
#Rename the sorbent to LLNL SCIE format
df5['redox'].replace({'I': '+1', 'II': '+2', 'III':'+3', 'IV':'+4', 'V':'+5' ,'VI':'+6', 'n.r':'n.r', 'VII':'+7', 'VIII':'+8', '-I':'-1','-II':'-2', 'V/IV': "+5,4", 'IV/V': "+4,5", 'V/VI': "+5,6"}, inplace=True)
df5['Sorbent']=df5['Element']+'('+df5['redox'].astype(str)+')'
df5['redox'].replace({'+1':'I', '+2':'II', '+3':'III', '+4':'IV', '+5':'V' ,'+6':'VI', 'n.r':'n.r', '+7':'VII', '+8':'VIII', '-1':'-I','-2':'-II', "+5,4":'V/IV',  "+4,5":'IV/V',  "+5,6":'V/VI'}, inplace=True)
df5.drop('Element', inplace=True, axis=1)
df5.drop('redox', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [9]:
#Export csv of all jaea subset database minerals and Rename Solid Phase related columns
df5.rename({'Solid Phase':'Mineral_sources'},axis=1,inplace=True)
df7 = pd.read_csv(path_1.joinpath("minerals_conversion.csv"))
df7 = df7.reset_index().set_index('Mineral_source')
df5['Mineral']=df5.apply(lambda row: df7.loc[row['Mineral_sources'],'Mineral'],axis=1)
df5['Mineral_formula']=df5.apply(lambda row: df7.loc[row['Mineral_sources'],'Mineral_formula'],axis=1)
df5.rename({'Mineral_sources':'Mineral_source'},axis=1,inplace=True)
#df5.to_csv('df5.csv')

In [10]:
#Rename surface area information
df5.rename({'Specific Surface Area(m2/g)':'MineralSA'},axis=1,inplace=True)
df5['temporaire']=df5['MineralSA'].str.find(')')
df5['MineralSA_SD']=df5.apply(lambda row: row['MineralSA'][row['temporaire']+1:] if row['temporaire']!=-1 else '',axis=1)
df5['MineralSA_SD'].replace({'':'NA'},inplace=True)
df5['temporaire']=df5['MineralSA'].str.find('(')
df5['MineralSA']=df5.apply(lambda row: row['MineralSA'][:row['temporaire']] if row['temporaire']!=-1 else row['MineralSA'][:],axis=1)
df5['MineralSA_units']='m2/g'
df5.drop('temporaire', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [11]:
#Rename CEC information
df5.rename({'CEC(meq/100g)':'CEC'},axis=1,inplace=True)
df5.replace({'':'NA','n.r.':'NA'},inplace=True)
df5.fillna('NA',inplace=True)
df5['CEC_SD']='NA'
df5['CEC_units']='meq/100g'
df5['temporaire']=df5['CEC'].str.find(')')
df5['CEC_SD']=df5.apply(lambda row: row['CEC'][row['temporaire']+1:] if row['temporaire']!=-1 else '',axis=1)
df5['CEC_SD'].replace({'':'NA'},inplace=True)
df5['temporaire']=df5['CEC'].str.find('(')
df5['CEC']=df5.apply(lambda row: row['CEC'][:row['temporaire']] if row['temporaire']!=-1 else row['CEC'][:],axis=1)
df5['CEC_units']='meq/100g'
df5.drop('temporaire', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [12]:
#Remove unwanted columns
df5.drop('note', inplace=True, axis=1)
df5.drop('Solid Phase Group', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [13]:
#Invert the solid solution value
df5['Mineral_SD']='NA'
df5['Mineral_units']='g/L'
def initial_fxn (the_input):
    if 'ca.' in the_input:
        return float(the_input[3:])
    elif '-' in the_input:
        a=float(the_input[:the_input.index('-')])
        b=float(the_input[the_input.index('-')+1:])
        return (b-a)/2
    else:
        return 'NA'

def the_fxn (my_input):
    try:
        return 1/float(my_input)*1000
    except:
        if 'ca.' in my_input:
            return 1/float(my_input[3:])*1000
        elif '-' in my_input:
            a=float(my_input[:my_input.index('-')])
            b=float(my_input[my_input.index('-')+1:])
            c=a+((b-a)/2)
            return 1/float(c)*1000
        else:
            return 'NA'

df5['Mineral_SD']=df5.apply(lambda row: initial_fxn(row['Solution/Solid(mL/g)']),axis=1)
df5['Solution/Solid(mL/g)']=df5.apply(lambda row: the_fxn(row['Solution/Solid(mL/g)']),axis=1)
df5.rename({'Solution/Solid(mL/g)':'Mineral_val'},axis=1,inplace=True)
#df5.to_csv('df5.csv')

In [14]:
#Export csv of all jaea subset database temperatures
df5.drop('Solution(mL)', inplace=True, axis=1)
df5.drop('Solid(g)', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [15]:
#Rename temperature info
def another_fxn(the_reference):
    if '+' in the_reference and '-' in the_reference:
        return float(the_reference[:the_reference.index('(')])
    elif '-' in the_reference:
        a=float(the_reference[:the_reference.index('-')])
        b=float(the_reference[the_reference.index('-')+1:])
        return float(a+b/2)
    elif ',' in the_reference:
        return 40
    elif '?' in the_reference:
        return float(the_reference[the_reference.index('?')-2:the_reference.index('?')])
    else:
        try:
            return float(the_reference)
        except:
            try:
                return float(the_reference[0:2])
            except:
                return 'NA'

def final_fxn(other_reference):
    if ')' in other_reference and 'room' not in other_reference:
        try:
            return float(other_reference[other_reference.index(')')+1:])
        except:
            return float(other_reference[-2])
    elif '-' in other_reference:
        a=float(other_reference[:other_reference.index('-')])
        b=float(other_reference[other_reference.index('-')+1:])
        return float(b-a/2)
    elif ',' in other_reference:
        return 20
    elif '}' in other_reference:
        return float(other_reference[other_reference.index('}')+1:other_reference.index('}')+2])
    else:
        try:
            return float(other_reference[-1])
        except:
            return 'NA'

df5['Temp']=df5.apply(lambda row: another_fxn(row['temp(degC)']),axis=1)
df5['Temp_SD']=df5.apply(lambda row: final_fxn(row['temp(degC)']),axis=1)
df5['Temp_SD'].replace({0:'NA', '0':'NA'},inplace=True)
df5.drop('temp(degC)', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [16]:

#Multistep conversion of electrolyte information
reference=[('Na(ppm)','Na(+1)',22.99),('K(ppm)','K(+1)',39.1),('Ca(ppm)','Ca(+2)',40.08),('Mg(ppm)','Mg(+2)',24.31),('Cl(ppm)','Cl(-1)',35.45),('NO3(ppm)','N(+5)',62.01),('ClO4(ppm)','Cl(+7)',99.45),('HCO3(ppm)','C(+4)',61.02),('SO4(ppm)','S(+6)',96.06),('F(ppm)','F(-1)',19.0),('SiO2(ppm)','Si(+4)',60.09),('Fe(ppm)','Fe(+3)',55.84),('DOC(ppm)','DOC',12.01)]

def a_fxn(a,b):
    if a=='NA':
        return 'NA'
    else:
        return b

for f in range(1,len(reference)+1):
    df5["Electrolyte"+str(f)+"_val"]=df5.apply(lambda row: row[reference[f-1][0]]/reference[f-1][2]/1000 if row[reference[f-1][0]] != 'NA' else 'NA',axis=1)
    df5["Electrolyte"+str(f)]=reference[f-1][1]
    df5["Electrolyte"+str(f)+"_SD"]='NA'
    df5["Electrolyte"+str(f)+"_units"]='mol/L'
    df5["Electrolyte"+str(f)]=df5.apply(lambda row: a_fxn(row["Electrolyte"+str(f)+"_val"], row["Electrolyte"+str(f)]), axis=1)
    df5["Electrolyte"+str(f)+"_units"]=df5.apply(lambda row: a_fxn(row["Electrolyte"+str(f)+"_val"], row["Electrolyte"+str(f)+"_units"]), axis=1)
    df5.drop(reference[f-1][0], inplace=True, axis=1)


df5.fillna('NA',inplace=True)
#df5.to_csv('df5.csv')

In [17]:
#Rename pH info and export csv with all pH values
df5['pH']=df5.apply(lambda row: row['pH end'] if row['pH end'] != 'NA' else row['pH init'],axis=1)
df5.drop('pH end', inplace=True, axis=1)
df5['pH_SD']='NA'
def pH_fxn(my_input):
    if 'as' in my_input:
        my_input=my_input[:-7]
    if '(' in my_input:
        return float(my_input[:my_input.index('(')])
    elif '-' in my_input:
        a=float(my_input[:my_input.index('-')])
        b=float(my_input[my_input.index('-')+1:])
        return (a+b)/2
    else:
        return my_input

def pH_SD_fxn(same_input,original):
    if 'as' in same_input:
        same_input=same_input[:-7]
    if ')' in same_input:
        return float(same_input[same_input.index(')')+1:])
    elif '-' in same_input:
        a=float(same_input[:same_input.index('-')])
        b=float(same_input[same_input.index('-')+1:])
        return (b-a)/2
    else:
        return original
df5['pH_SD']=df5.apply(lambda row: pH_SD_fxn(row['pH'],row['pH_SD']),axis=1)
df5['pH']=df5.apply(lambda row: pH_fxn(row['pH']),axis=1)
#df5.to_csv('df5.csv')

In [18]:
#Rename Sorbent concentration info and export csv of all concentration data
def sorbent_SD_fxn(my_input):
    if '<' in my_input:
        return float(my_input[1:])
    elif '(1.2-1.3)' in my_input:
        return 0.05*10**(-7)
    elif ' - ' in my_input:
        a=float(my_input[:my_input.index(' - ')])
        b=float(my_input[my_input.index(' - ')+3:])
        return (b-a)/2
    else:
        return my_input

def sorbent_fxn(same_input):
    if '<' in same_input:
        return float(same_input[1:])
    elif '(1.2-1.3)' in same_input:
        return 1.25*10**(-7)
    elif ' - ' in same_input:
        a=float(same_input[:same_input.index(' - ')])
        b=float(same_input[same_input.index(' - ')+3:])
        return (a+b)/2
    elif 'and' in same_input:
        return 'NA'
    else:
        return same_input

df5.rename({'C init(mol/L)':'Sorbent_val'},axis=1,inplace=True)
df5['Sorbent_SD']='NA'
df5['Sorbent_SD']=df5.apply(lambda row: sorbent_SD_fxn(row['Sorbent_val']),axis=1)
df5['Sorbent_val']=df5.apply(lambda row: sorbent_fxn(row['Sorbent_val']),axis=1)
df5['Sorbent_units']='mol/L'
#df5.to_csv('df5.csv')

In [19]:
#Remove columns
df5.drop('Contact time(day)', inplace=True, axis=1)
df5.drop('Separation', inplace=True, axis=1)
df5.drop('Eh init(mV)', inplace=True, axis=1)
df5.drop('Eh end(mV)', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [20]:
#Multistep conversion of gas composition info
df6 = pd.read_csv(path_1.joinpath("gas_composition_conversion.txt"),sep = '\t')
df6 = df6.reset_index().set_index('jaea: atm/redox condition')
other_reference=['Gas1','Gas1_val','Gas1_SD','Gas1_units','Gas2','Gas2_val','Gas2_SD','Gas2_units','Gas3','Gas3_val','Gas3_SD','Gas3_units']
df5.fillna('NA',inplace=True)
for g in other_reference:
    df5[g]=df5.apply(lambda row: df6.loc[row['atm/redox condition'],g] if row['atm/redox condition'] != 'NA' else 'NA',axis=1)
#df5.to_csv('df5.csv')

In [21]:
#Remove columns
df5.drop('Note', inplace=True, axis=1)
df5.drop('replicates, n', inplace=True, axis=1)
df5.drop('additional/information', inplace=True, axis=1)
df5.drop('Kd Graph Data', inplace=True, axis=1)
df5.drop('Temp Graph Data', inplace=True, axis=1)
df5.drop('pH Graph Data', inplace=True, axis=1)
df5.drop('Time Graph Data', inplace=True, axis=1)
df5.drop('Eh Graph Data', inplace=True, axis=1)
df5.drop('Cinit Graph Data', inplace=True, axis=1)
df5.drop('Solution/Solid Graph Data', inplace=True, axis=1)
df5.drop('Data type', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [22]:
#Multistep conversion of jaea: kd column
def last_fxn(my_input):
    a=float(my_input[:my_input.index(' - ')])
    b=float(my_input[my_input.index(' - ')+3:])
    return (a+b)/2

df5['check']=df5.apply(lambda row: True if ('>' in row['Kd(m3/kg)'] or '<' in row['Kd(m3/kg)']) else False,axis=1)
df5['Kd(m3/kg)']=df5.apply(lambda row: float(row['Kd(m3/kg)'][1:]) if row['check'] is True else row['Kd(m3/kg)'],axis=1)
df5['Kd(m3/kg)']=df5.apply(lambda row: last_fxn(row['Kd(m3/kg)']) if ' - ' in str(row['Kd(m3/kg)']) else row['Kd(m3/kg)'],axis=1)
df5['error'].replace({'?}0.2log':0.2, '0.5log':0.5, '-':'NA'},inplace=True)
df5['error']= df5.apply(lambda row: 0.05*float(row['Kd(m3/kg)']) if (row['error']=='?`5%' or row['error']=='5%') else row['error'],axis=1)
df5['Aq_val'] = df5.apply(lambda row: float(row['Sorbent_val']) / (float(row['Kd(m3/kg)']) * float(row['Mineral_val']) + 1) if ((row['Sorbent_val']) != 'NA' and (row['Kd(m3/kg)']) != 'NA' and (row['Mineral_val']) != 'NA') else 'NA',axis=1)
df5['Aq_SD'] = df5.apply(lambda row: ((float(row['Sorbent_val']) / (float(row['Kd(m3/kg)']) * float(row['Sorbent_val']) + 1)) - (float(row['Sorbent_val']) / ((float(row['Kd(m3/kg)']) + float(row['error'])) * float(row['Sorbent_val']) +1))) if ((row['Sorbent_val']) != 'NA' and (row['Kd(m3/kg)']) != 'NA' and (row['Mineral_val']) != 'NA' and row['error'] != 'NA' and row['check'] is False) else (row['Kd(m3/kg)'] if row['check'] is True else 'NA'),axis=1)
df5['Aq_units']='mol/L'
df5['Sorbed_val']=df5.apply(lambda row: float(row['Sorbent_val'])-float(row['Aq_val']) if ((row['Sorbent_val']) != 'NA' and (row['Aq_val']) != 'NA') else 'NA',axis=1)
df5['Sorbed_SD']=df5.apply(lambda row: math.sqrt((float(row['Sorbent_SD'])*float(row['Sorbent_SD']))+(float(row['Aq_SD'])*float(row['Aq_SD']))) if ((row['Sorbent_SD']) != 'NA' and (row['Aq_SD']) != 'NA') else 'NA',axis=1)
df5['Sorbed_units']='mol/L'
df5.drop('Kd(m3/kg)', inplace=True, axis=1)
df5.drop('error', inplace=True, axis=1)
df5.drop('check', inplace=True, axis=1)
df5.to_csv('df5.csv')
